# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [1]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [2]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [3]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [4]:
from isadream.isadream import modelUtils
from isadream.isadream import io
from isadream.isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [5]:
# A demo json file is provided.
nmr_json_demo = modelUtils.SIPOS_DEMO
demo_base_path = modelUtils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tyler/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tyler/git/isadream/isadream/demo_data


---

In [17]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

In [27]:
# node

In [28]:
help(node)

Help on DrupalNode in module isadream.isadream.models.nodal object:

class DrupalNode(param.parameterized.Parameterized)
 |  params(assays=List, comments=List, factors=List, info=Dict, samples=List, title=String, name=String)
 |  Model for a single Drupal content node.
 |  
 |      Much of this class is declarative, with the Param package doing all of the
 |      heavy-lifting in the background.
 |  
 |      
 |  Parameters of 'DrupalNode'
 |  ==========================
 |  
 |  Parameters changed from their default values are marked in red.
 |  Soft bound values are marked in cyan.
 |  C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None
 |  
 |  Name      Value   Type     Bounds     Mode  
 |  
 |  assays      []    List   (0, None)  V RW AN 
 |  comments    []    List   (0, None)  V RW AN 
 |  factors     []    List   (0, None)  V RW AN 
 |  info       None   Dict              V RW AN 
 |  samples     []    List   (0, None)  V RW AN 
 |  title       ''   String         

In [7]:
import param

In [10]:
help(param.Parameterized)

Help on class Parameterized in module param.parameterized:

class Parameterized(builtins.object)
 |  params(name=String)
 |  
 |      Base class for named objects that support Parameters and message
 |      formatting.
 |  
 |      Automatic object naming: Every Parameterized instance has a name
 |      parameter.  If the user doesn't designate a name=<str> argument
 |      when constructing the object, the object will be given a name
 |      consisting of its class name followed by a unique 5-digit number.
 |  
 |      Automatic parameter setting: The Parameterized __init__ method
 |      will automatically read the list of keyword parameters.  If any
 |      keyword matches the name of a Parameter (see Parameter class)
 |      defined in the object's class or any of its superclasses, that
 |      parameter in the instance will get the value given as a keyword
 |      argument.  For example:
 |  
 |        class Foo(Parameterized):
 |           xx = Parameter(default=1)
 |  
 |       

In [9]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,sample_node,27 Al ppm,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration
0,75a7c061-dbe7-4573-8870-cbdc94c78ee0,26ae9305-ec98-490a-b231-d09b348f271a,f785974e-fb3f-4a64-bf13-f38c273837a9,79.90,2.93,2.93,K+,2.93
1,75a7c061-dbe7-4573-8870-cbdc94c78ee0,26ae9305-ec98-490a-b231-d09b348f271a,f785974e-fb3f-4a64-bf13-f38c273837a9,79.84,4.92,4.92,K+,4.92
2,75a7c061-dbe7-4573-8870-cbdc94c78ee0,26ae9305-ec98-490a-b231-d09b348f271a,f785974e-fb3f-4a64-bf13-f38c273837a9,79.72,6.85,6.85,K+,6.85
3,75a7c061-dbe7-4573-8870-cbdc94c78ee0,26ae9305-ec98-490a-b231-d09b348f271a,f785974e-fb3f-4a64-bf13-f38c273837a9,79.66,9.13,9.13,K+,9.13
4,75a7c061-dbe7-4573-8870-cbdc94c78ee0,26ae9305-ec98-490a-b231-d09b348f271a,f785974e-fb3f-4a64-bf13-f38c273837a9,79.66,10.71,10.71,K+,10.71
0,8d74bf04-feff-4890-8fee-3df96f08fa5f,c1f08b80-59de-4ace-b6db-890332b22b2f,8c26c344-40b2-4cf0-87bf-68170ed7f92e,79.92,0.66,0.66,Li+,0.66
1,8d74bf04-feff-4890-8fee-3df96f08fa5f,c1f08b80-59de-4ace-b6db-890332b22b2f,8c26c344-40b2-4cf0-87bf-68170ed7f92e,79.84,1.10,1.10,Li+,1.10
2,8d74bf04-feff-4890-8fee-3df96f08fa5f,c1f08b80-59de-4ace-b6db-890332b22b2f,8c26c344-40b2-4cf0-87bf-68170ed7f92e,79.66,1.64,1.64,Li+,1.64
3,8d74bf04-feff-4890-8fee-3df96f08fa5f,c1f08b80-59de-4ace-b6db-890332b22b2f,8c26c344-40b2-4cf0-87bf-68170ed7f92e,79.54,2.14,2.14,Li+,2.14
4,8d74bf04-feff-4890-8fee-3df96f08fa5f,c1f08b80-59de-4ace-b6db-890332b22b2f,8c26c344-40b2-4cf0-87bf-68170ed7f92e,79.36,2.59,2.59,Li+,2.59


In [21]:
metadata = [md for _, md in node_assays]
metadata
md = collections.ChainMap(*metadata)
# dict(metadata)

NameError: name 'node_assays' is not defined

In [25]:
md.keys()

KeysView(ChainMap(defaultdict(<class 'tuple'>, {'7bc0a784-2f8f-44ba-9041-a7e0d9812f89': ({'$id': 'https://lampdev02.pnl.gov/bigg006/idreamdrupal/', 'title': 'Sipos 2006, Talanta NMR Figures', 'filename': 'sipos_2006_talanta_nmr_figs.json', 'description': 'Extracted figures.', 'submissionDate': '2018-5-25', 'publicReleaseDate': '2006-03-13', 'experimentSubType': 'Al_NMR'}, [None]), 'e1988d5d-1cc3-4134-baa4-d164caf86b79': (NodeInfo(info={'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'}, name='NodeInfo00075'), [None]), '2a7e4c6b-864d-411a-8cff-a6b8e465e440': {'sampleName': 'Aluminate Solution'}, '18c201f8-cfd5-4d67-b82e-4d53d24dc1d8': {'sampleName': 'Aluminate Solution'}, '977d884f-a11b-4bb2-b4ef-3b25f966cc40': {'sampleName': 'Potassium Hydroxide'}, '728e2f7c-e1b0-4a80-b189-ec24c63a49bf': {'sampleName': 'Potassium Hydroxide'}, 'e1025e99-bafc-43f9-898a-bab9e40045fa': {'sampleName': 'Potassium Hydroxide'}}), defaultdict(<class 'tuple'>, {'9d992d88-f405-4481-94df-b67991dabf6b': ({'$id': 'http

---